<a href="https://colab.research.google.com/github/rahiakela/practical-machine-learning-with-tensorflow/blob/week-5/convolutional_neural_networks_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks Part-1

This tutorial demonstrates training a simple [Convolutional Neural Network](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) to classify MNIST digits. This simple network will achieve over 99% accuracy on the MNIST test set. Because this tutorial uses the [Keras Sequential API](https://www.tensorflow.org/guide/keras), creating and training our model will take just a few lines of code.

Note: CNNs train faster with a GPU. If you are running this notebook with Colab, you can enable the free GPU via * Edit -> Notebook settings -> Hardware accelerator -> GPU*.

### Import TensorFlow

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

TensorFlow 2.x selected.


### Download and prepare the MNIST dataset

In [3]:
(train_images, train_labels),(test_images, test_labels) = datasets.mnist.load_data()

print('Before reshaping:')
print('##################')
print('Number of axes in train_images:', train_images.ndim)
print('Number of axes in test_images:', test_images.ndim)
print('Shape in train_images:', train_images.shape)
print('Shape in test_images:', test_images.shape)
print()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

print('After reshaping:')
print('##################')
print('Number of axes in train_images:', train_images.ndim)
print('Number of axes in test_images:', test_images.ndim)
print('Shape in train_images:', train_images.shape)
print('Shape in test_images:', test_images.shape)
print()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

Before reshaping:
##################
Number of axes in train_images: 3
Number of axes in test_images: 3
Shape in train_images: (60000, 28, 28)
Shape in test_images: (10000, 28, 28)

After reshaping:
##################
Number of axes in train_images: 4
Number of axes in test_images: 4
Shape in train_images: (60000, 28, 28, 1)
Shape in test_images: (10000, 28, 28, 1)



### Create the convolutional base

The 6 lines of code below define the convolutional base using a common pattern: a stack of [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) and [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to color channels, MNIST has one (because the images are grayscale), whereas a color image has three (R,G,B). In this example, we will configure our CNN to process inputs of shape (28, 28, 1), which is the format of MNIST images. We do this by passing the argument `input_shape` to our first layer.

